# Lumber Transportation Problem 
J. Reeb and S. Leavengood

Millco has three wood mills and is planning three new logging sites. Each mill has a maximum capacity and each logging site can harvest a certain number of truckloads of lumber per day. The cost of a haul is $2/mile of distance. If distances from logging sites to mills are given below, how should the hauls be routed to minimize hauling costs while meeting all demands?

| Logging Site | Mill A | Mill B | Mill C | Max loads per day |
|:------------:|:------:|:------:|:------:|:-----------------:|
| 1            |  8     |  15    |  50    |  20               |
| 2            |  10    |  17    |  20    |  30               |
| 3            |  30    |  26    |  15    |  45               |
| Mill demand  |  30    |  35    |  30    |                   |

In [1]:
from gamspy import Container, Sum, Sense
import numpy as np

In [2]:
m = Container()

# Model sets and parameters
sites = m.addSet("sites", records=["1", "2", "3"])
mills = m.addSet("mills", records=["Mill A", "Mill B", "Mill C"])
dist = m.addParameter(
    "dist", [sites, mills], records=np.array([[8, 15, 50], [10, 17, 20], [30, 26, 15]])
)
supply = m.addParameter("supply", [sites], records=np.array([20, 30, 45]))
demand = m.addParameter("demand", [mills], records=np.array([30, 35, 30]))
cost_per_haul = 4

# Model variables
m.addAlias("s", sites)
m.addAlias("m", mills)
ship = m.addVariable("ship", "positive", [sites, mills])

defcost = cost_per_haul * Sum([sites, mills], ship[sites, mills] * dist[sites, mills])

defsupply = m.addEquation(name='defsupply', domain=sites)
defsupply[sites] = Sum(mills, ship[sites, mills]) == supply[sites]

defdemand = m.addEquation(name='defdemand', domain=mills)
defdemand[mills] = Sum(sites, ship[sites, mills]) == demand[mills]

millco = m.addModel(
    name='millco',
    equations=m.getEquations(),
    problem='LP',
    sense=Sense.MIN,
    objective=defcost
)

In [3]:
millco.solve()

In [4]:
ship.records.pivot(index='sites', columns='mills', values='level')

mills,Mill A,Mill B,Mill C
sites,,,
1,0.0,20.0,0.0
2,30.0,0.0,0.0
3,0.0,15.0,30.0


In [5]:
print(f'Total cost will be {millco.objective_value}')

Total cost will be 5760.0


In [6]:
m = Container()

# Generic Model sets and parameters
i = m.addSet('i', records = ['i'+str(i) for i in range(6)], description = 'equation index')
j = m.addSet('j', records = ['j'+str(j) for j in range(9)], description = 'variable index')
A = m.addParameter('A', [i,j], records = np.array([[1,1,1,0,0,0,0,0,0],
                                                   [0,0,0,1,1,1,0,0,0],
                                                   [0,0,0,0,0,0,1,1,1],
                                                   [1,0,0,1,0,0,1,0,0],
                                                   [0,1,0,0,1,0,0,1,0],
                                                   [0,0,1,0,0,1,0,0,1]]))
b = m.addParameter('b', [i], records = np.array([20,30,45,30,35,30]))
c = m.addParameter('c', [j], records = np.array([8,15,50,10,17,20,30,26,15]))
x = m.addVariable('x', 'positive', [j])

obj = 4 * Sum(j, c[j] * x[j])

e = m.addEquation(name='e', domain=i)
e[i] = Sum(j, A[i,j] * x[j]) == b[i]

generic_model = m.addModel(
    name='generic_model',
    equations=m.getEquations(),
    problem='LP',
    sense=Sense.MIN,
    objective=obj
)

In [7]:
generic_model.solve()

In [8]:
x.records

,j,level,marginal,lower,upper,scale
0,j0,0.0,-0.0,0.0,inf,1.0
1,j1,20.0,0.0,0.0,inf,1.0
2,j2,0.0,184.0,0.0,inf,1.0
3,j3,30.0,0.0,0.0,inf,1.0
4,j4,0.0,0.0,0.0,inf,1.0
5,j5,0.0,56.0,0.0,inf,1.0
6,j6,0.0,44.0,0.0,inf,1.0
7,j7,15.0,0.0,0.0,inf,1.0
8,j8,30.0,0.0,0.0,inf,1.0


In [9]:
print(f'Total cost will be {generic_model.objective_value}')

Total cost will be 5760.0
